### Here we define the calculations we'll use in order to characterise the mobility of the participants, in particular we focus on velocities. First we compute the time difference between Geo-locations in seconds. Then we obtain the distance in metres between Geo-locations and the instantaneous velocity, defined as the distance over time difference. Finally, we show how to obtain the Mean Squared Displacement for two cases and the Autocorrelation (of the logarithm of the velocities)

# INDEX

    1. Time difference between Geo-locations in seconds
    
    2. Euclidean distance between Geo-locations in metres
        2.1- Harversine Equation
        2.2- UTM projection. Euclidean distance
        
    3. Instantaneous velocity
        
    4. Mean Squared Displacement MSD(T)
        4.1- Heterogenous case (origin at t=0)
        4.2- Homogenous case (averaging over all origins, thermalising)
    
    5. Auto-correlation function
    
    6. Tortuosity and Reorientation
   

In [1]:
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os
from math import sin, cos, sqrt, atan2, radians
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

#%matplotlib inline
ox.config(log_console=True)
ox.__version__




'1.0.1'

# 1. Time difference between Geo-locations in seconds

In [2]:
def TimeDifference(df,name_column_time):
    """ Given the dataframe and the name of the column that contains the time of each Geo-location, it returns the dataframe
        with a new column corresponding to the time difference between geolocations (so the last element is null)
        
        Inputs:
            - Dataframe
            - name of the time column (string). Eg: 'time'
        
        Outputs:
            - Dataframe with a new column corresponding to the time difference between consecutive geolocations
    """ 
    
    
    df[name_column_time] = pd.to_datetime(df[name_column_time], format='%Y-%m-%d %H:%M:%S')  # Time column to datetime format
    df=df.drop_duplicates(subset=[name_column_time],keep='first')  # drop duplicates (impossible)
    
    # Compute the time difference between consecutive geolocations [in advance: At(i)=t(i+1)-t(i)]
    time=df[name_column_time].tolist()   
    diff_time=[]
    for i in range(1,len(df[name_column_time])):
        diff=temps[i]-temps[i-1]
        diff_time.append(diff)

    diff_time.insert(len(diff_time), np.nan)   # Add NaN value at the end of the list to add as a column in the dataframe
                                               # (since we have N data and N-1 time differences)
 
    df['At']=diff_time     # Add new column At corresponding to the time difference in seconds
    df['At']=df['At'].dt.seconds
    
    return df





# 2. Euclidian distance between Geo-locations in metres

In this work we'll use the Way 1, using the Harvensine Equation 

## 2.1- Harvensine Equation 

In [4]:
def getDistanceFromLatLonInM(lat1,lon1,lat2,lon2):
    """ Function that returns the distance in metres between 2 GPS locations in degrees (latitude and longitude).
    It is based in the Haversine formula (https://en.wikipedia.org/wiki/Haversine_formula) which takes into account the
    Earth's curvature. 
    
    Input:
        - 2 GPS coordinates: (latitude1,longitude1) of the first point and (latitude2,longitude2) of the second point. 
        
    Output:
        - Distance in metres between the two GPS locations.
    """
    
    R = 6371 # Radius of the earth in km
    dLat = radians(lat2-lat1)
    dLon = radians(lon2-lon1)
    rLat1 = radians(lat1)
    rLat2 = radians(lat2)
    a = sin(dLat/2) * sin(dLat/2) + cos(rLat1) * cos(rLat2) * sin(dLon/2) * sin(dLon/2) 
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c # Distance in km
    e= d*1000 #distance in m
    
    return e



    
    
    

## 2.2- UTM projection. Euclidean distance

In [6]:
import utm

def GPScoordinates_to_utm(lat,lon):
    """" Function that projects the GPS coordinates in degrees (latitude, longitude) into the UTM coordinate system
    in order to work with the concept of "point" and "Euclidean distance" in a plane.
    (https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system)
    
    Note: We are dealing with locations in the same area/region. Otherwise, we should be careful if two locations 
    belong to different UTM zones when calculating distances, etc. 
    
    Input:
        - lists of GPS coordinates: latitude and longitude
        
    Output:
        - lists of the UTM projections of the GPS coordinates: 
        - The utm package returns Easting, Northing, Zone_number and Zone_letter. So we only store the two first elements
    """
    
    lat_utm=[]
    lon_utm=[]
    for i in range(len(lat)):
        u=utm.from_latlon(lat[i],lon[i])  # get the UTM projection
        lat_utm.append(u[0])   # Store the projection of latitude and longitude in lists
        lon_utm.append(u[1])
        
    return lat_utm, lon_utm



def EuclidianDistance(x1,y1,x2,y2):
    """ Function that returns the distance in metres between 2 points in a plane (NO GPS locs.) 
    
    Note: Be careful as the GPS coordinates (lat,lon) cannot be used, but their projections to a plane (e.g. utm projection).

    
    Input:
        - The coordinates of two points in a plane: (x1,y1) and (x2,y2).
        
    Output:
        - Euclidian distance in metres between the two points.
    """
    
    Euclidian_distance = ( (x2-x1)**2 + (y2-y1)**2 ) ** 0.5
        
    return Euclidian_distance

    
    

# 3. Instantaneous velocity

In [7]:
def instantaneous_velocity(distance,time):
    """" Function that computes the instantenous velocity between two points, given their distance and their time difference.
    
    Input:
        - Distance between two points or two locations
        - Time difference between the two points/locations
        
    Output:
        - Instantenous velocity between the two points
    """
    
    v=distance/time
    
    return v

# 4. Mean Squared Displacement

$\Large MSD(T)=\langle |\vec{r}(t+T)-\vec{r}(t)|^2 \rangle $

## 4.1- Heterogeneous case (origin at t=0)

In [ ]:
""" Obtain the Mean Squared Displacement, averaged over all users (for each time "s", averaging over users) and taking
    t=0 as the origin (heterogeneous case). 
    
    Inputs:
        - Dataframe
        
    Outputs:
        - text files with the MSD(T) for each T and confident intervals of 95%. And the plot of MSD(T).
    """


all_files = glob.glob(os.path.join("*.csv")) #make list of paths for all the csv files (each user)

diff_mean_n_square=[]
interv_conf_95_plus=[]
interv_conf_95_minus=[]

for s in range(600):   #loop for each time T (T=1,2,3,...,600)

    diff_list2 = [] 

    for file in all_files:   #loop for each user file (csv). 
        df = pd.read_csv(file) #read the file

        latitude = df['latitude'].tolist()   #latitude and longitude to lists
        longitude = df['longitude'].tolist()

        if len(latitude) > s:  #use only those files of lenght > s to compute the MSD (because every user has different lenght)

            x=latitude[0]
            y=longitude[0]
            z=latitude[s]
            t=longitude[s]                          #|r(s)-r(0)|^2
            diff=getDistanceFromLatLonInM(x,y,z,t)  #computes the distances between the origin (t=0) and the position at "s"
            diff_list2.append(abs(diff)**2)    #append in a list all the distances (squared) # |r(s)-r(0)|^2 for each user


    average_diff2=sum(diff_list2)/len(diff_list2)  # <|r(s)-r(0)|^2> average over users for a given s
    diff_mean_n_square.append(average_diff2)  #we store the averaged result over users for a every s

    mean = average_diff2   #mean, variance and standard deviation for the interval confidence
    variance = sum([((xx - mean) ** 2) for xx in diff_list2]) / len(diff_list2)
    stddev = variance ** 0.5
    interv_conf_95_plus.append(mean+((1.960*stddev)/((len(diff_list2))**0.5)))   #interval confidence of 95% (z=1.960)
    interv_conf_95_minus.append(mean-((1.960*stddev)/((len(diff_list2))**0.5)))


list_s=[]
for i in range(600):
    list_s.append(i)

#PLOT THE RESULT FOR THE MSD   
fig, ax1 = pp.subplots(figsize=(7,5))   
ax1.plot(list_s, diff_mean_n_square, '+', label='ordered') 
ax1.fill_between(list_s, interv_conf_95_minus, interv_conf_95_plus, color='b', alpha=.1)
ax1.set_ylabel(r'MSD ($m^2$)',fontsize=14)
ax1.set_xlabel(r'T (s)',fontsize=14)
ax1.set_yscale('log',basey=10) 
ax1.set_xscale('log',basex=10) 
axins2 = inset_axes(ax1, width="80%", height="100%", loc=1,bbox_to_anchor=(0.10,1-0.45,.35,.35), bbox_transform=ax1.transAxes)
axins2.plot(list_s, diff_mean_n_square,'+')
axins2.fill_between(list_s, interv_conf_95_minus, interv_conf_95_plus, color='b', alpha=.1)


# SAVE TO FILES. 

#np.savetxt('difussion_origin_0.txt', np.array(diff_mean_n_square))
#np.savetxt('CI_95_origin_0_plus.txt',np.array(interv_conf_95_plus))
#np.savetxt('CI_95_origin_0_minus.txt',np.array(interv_conf_95_minus))

pp.show()

## 4.2- Homogenous case (averaging over all origins, thermalising)

In [ ]:
""" Obtain the Mean Squared Displacement, averaged over all users (for each time "s", averaging over users) and also averaging
    over all possible origins (thermalising). 
    
    Inputs:
        - Dataframe
        
    Outputs:
        - text files with the MSD(T) for each T and confident intervals of 95%. And the plot of MSD(T).
    """


all_files = glob.glob(os.path.join("*.csv")) #make list of paths for all the csv files (each user)

diff_mean_s=[]
diff_mean_s_square=[]
interv_conf_95_plus=[]
interv_conf_95_minus=[]

for s in range(1,600):   #loop for each T (T=1,2,3,...,200)
    diff_mean_n=[]
    diff_mean_n_square=[]
    
    for file in all_files:   #loop for each user file (csv). 
        df = pd.read_csv(file) #read the file
        
        latitude = df['latitude'].tolist()   #latitude and longitude to lists
        longitude = df['longitude'].tolist()

        if len(latitude) > s:   #use only those files of lenght > 200 to compute the MSD (almost all of them)
            
            diff_list2 = [] 
            
            for x, y, z, t in zip(latitude[0::], longitude[0::], latitude[s::], longitude[s::]): #|r(t+s)-r(t)|^2
                diff=getDistanceFromLatLonInM(x,y,z,y)  #computes ALL the distances between locations separated "s" timesteps
                diff_list2.append(diff**2)    #append in a list all the distances (squared) # |r(t+s)-r(t)|^2
                                              #for instance, for s=1:  r1-r0, r2-r1, r3-r2, r4-r1 (all 1 step distances)
                                              #for s=2: r2-r0, r3-r1, r4-r2,...etc (for a given individual)
      
            average_diff2=sum(diff_list2)/len(diff_list2)  # <|r(t+s)-r(t)|^2> average over t (for a given s and a given indiv.)
            diff_mean_n_square.append(average_diff2)  #we store the averaged result for each individual in a list (for a given s)
    
    diff_mean_n_square2=sum(diff_mean_n_square)/len(diff_mean_n_square) #Average over users, for a given s.
    diff_mean_s_square.append(diff_mean_n_square2)  #Then for each s, we store the averaged value of users. MSD(s)
    mean = diff_mean_n_square2    #mean, variance and st8andard deviation for the interval confidence
    variance = sum([((xx - mean) ** 2) for xx in diff_mean_n_square]) / len(diff_mean_n_square)
    stddev = variance ** 0.5
    interv_conf_95_plus.append(mean+((1.960*stddev)/((len(diff_mean_n_square))**0.5)))   #interval confidence of 95% (z=1.960)
    interv_conf_95_minus.append(mean-((1.960*stddev)/((len(diff_mean_n_square))**0.5)))

list_s=[]
for i in range(1,600):
    list_s.append(i)

#PLOT THE RESULT FOR THE MSD   
fig, ax1 = pp.subplots(figsize=(7,5))   
ax1.plot(list_s, diff_mean_s_square, '+', label='ordered') 
ax1.fill_between(list_s, interv_conf_95_minus, interv_conf_95_plus, color='b', alpha=.1)
ax1.set_ylabel(r'MSD ($m^2$)',fontsize=14)
ax1.set_xlabel(r'T (s)',fontsize=14)
ax1.set_yscale('log',basey=10) 
ax1.set_xscale('log',basex=10) 
axins2 = inset_axes(ax1, width="80%", height="100%", loc=1,bbox_to_anchor=(0.10,1-0.45,.35,.35), bbox_transform=ax1.transAxes)
axins2.plot(list_s, diff_mean_s_square,'+')

#np.savetxt('difussion_homogenous.txt', np.array(diff_mean_s_square))
#np.savetxt('ci_95_msd_homogenous_plus.txt', np.array(interv_conf_95_plus))
#np.savetxt('ci_95_msd_homogenous_minus.txt', np.array(interv_conf_95_minus))

pp.show()

# 5. Autocorrelation function

$\Large u=\ln \left( \frac{v}{\langle v \rangle} \right)$

$\Large C(\tau) = \frac{\langle \left[u(t)-\langle u(t) \rangle \right] \left[ u(t+\tau)-\langle u(t) \rangle \right] \rangle}{\langle \left[ u(t)-\langle u(t) \rangle \right]^{2} \rangle }$

In [ ]:
""" Obtain the Autocorrelation function C(tau), averaged over all users (for each time "tau", averaging over users). Using the 
    logarithm of the velocity (u=log(u/<v>)) using <v> a scale factor just for units.
    
    Inputs:
        - Dataframe
        
    Outputs:
        - text files with the C(tau) for each tau. And plot of C(tau)
    """


all_files = glob.glob(os.path.join("*.csv")) #make list of paths

#SAME CODE BUT NOW FOR S=1,...,600. SAVING THE AUTOCORRELATION VALUES IN A TEXT FILE.
corr_mean_s=[]
interv_conf_95_plus_log=[]
interv_conf_95_minus_log=[]
for s in range(1,600):   
    corr_mean_n=[]
    print(s)
    for file in all_files: 
        df = pd.read_csv(file)      
        
        v=df['v'][:-1].tolist()   # velocity to list and obtain u=ln(v/<v>) for each user (or the variable we want to study)
        mean_v=sum(v)/len(v)
        u=[]
        for i in range(len(v)):
            v_log=np.log(v[i]/mean_v)
            u.append(v_log)
            
        
        if len(u) > s:   # Compute autocorrelation and average over all s those users ho has a size >s (obviuosly). 
            mean=sum(u)/len(u)
            variance = sum([((y - mean) ** 2) for y in u]) / len(u)
            std= variance ** 0.5  

            delta_r_i_rescaled = [(x - mean)/std for x in u]
            
            res = tuple( i*j for i, j in zip(delta_r_i_rescaled, delta_r_i_rescaled[s:])) 
            average_corr=sum(res)/len(res)    
            corr_mean_n.append(average_corr)
            
    
    corr_mean_n2=sum(corr_mean_n)/len(corr_mean_n)
    corr_mean_s.append(corr_mean_n2)
    
    mean = corr_mean_n2   #mean, variance and standard deviation for the interval confidence
    variance = sum([((xx - mean) ** 2) for xx in corr_mean_n]) / len(corr_mean_n)
    stddev = variance ** 0.5
    interv_conf_95_plus_log.append(mean+((1.960*stddev)/((len(corr_mean_n))**0.5)))   #interval confidence of 95% (z=1.960)
    interv_conf_95_minus_log.append(mean-((1.960*stddev)/((len(corr_mean_n))**0.5)))
    
autocorr=np.array(corr_mean_s)

list_s=[]
for i in range(1,600):
    list_s.append(i)

    
#PLOT THE RESULT FOR THE C(tau)
fig, ax1 = pp.subplots(figsize=(7,5))   
ax1.plot(list_s, corr_mean_s, '+') 
ax1.set_ylabel(r'C($\tau$)',fontsize=14)
ax1.set_xlabel(r'$\tau$ (s)',fontsize=14)
ax1.set_xlim(0,202)
axins2 = inset_axes(ax1, width="80%", height="100%", loc=1,bbox_to_anchor=(0.10,1-0.45,.35,.35), bbox_transform=ax1.transAxes)
axins2.plot(list_s, corr_mean_s,'+')
axins2.set_yscale('log',basey=10) 
axins2.set_xscale('log',basex=10) 



#np.savetxt('autocorrelation_log.txt', autocorr)   # save to text
#np.savetxt('autocorrelation__CI_plus_log.txt', interv_conf_95_plus_log)
#np.savetxt('autocorrelation_CI_minus_log.txt', interv_conf_95_minus_log)





# 6. Tortuosity and Reorientation



$ \Large T = 1 - <cos(\theta)>$


being $\theta$ the re-orientation angle between the instantaneous vector of movement connecting the points $i$ and $i+1$ and the vector towards the final destination, connecting the points $i$ and $N$. 

In [1]:
from scipy.special import lpmn
import pandas as pd
import numpy as np
#from mpmath import *
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy.optimize import curve_fit
from scipy.special import lpmn,jv,lpmv
import networkx as nx
import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import glob
import os
from math import sin, cos, sqrt, atan2, radians
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.optimize import curve_fit
import scipy
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.ticker import FuncFormatter, MultipleLocator


def orientation(latitude0,longitude0,latitude1,longitude1):
    """ Given two points that form a vector: origin p0 and destination p1, get their angle (uniquely defined)
    
    Input:
        - origin latitude and longitude (latitude0, longitude0)
        - destination latitude and longitude (latitude1, longitude1)
    
    Output:
        - orientation angle between the vector and the x-coordenate (in radians)
    
    Note: The returned angle is in the range between 0 and 2·pi in counter-clockwise direction
    
    """
    
    p0=(latitude0,longitude0)   #first define the points: p0=(latitude0, longitude0) and p1=(latitude1,longitude1)
    p1=(latitude1,longitude1)
    p=np.array(p1)-np.array(p0)
    
    a=np.arctan2(p[1],p[0])         # orientation angle between 0 and 2·pi
    if a < 0.0:
        b=2*np.pi + a
        return b
    else:
        return a


    
    
def vector(latitude0,longitude0,latitude1,longitude1): 
    """ Given two points: origin p0 and destionation p1, get their vector coordenates 
    
    Input:
        - origin latitude and longitude (latitude0, longitude0)
        - destination latitude and longitude (latitude1, longitude1)
        
    Output:
        - vector coordinates between origin and destination points
    
    """
    
    p0=(latitude0,longitude0)   # first define the points: p0=(latitude0, longitude0) and p1=(latitude1,longitude1)
    p1=(latitude1,longitude1)
    vec=(p1[0]-p0[0], p1[1]-p0[1]) #vector
    return vec




def determinant(vec0,vec1):
    """ Given two vectors, compute the determinant. If det<0 means that the second vector has turned in the clockwise direction
    
    Input:
        - two consecutive vectors vec0 and vec1
        
    Output:
        - determinant
    """
    
    det=vec0[0]*vec1[1]-vec0[1]*vec1[0]
    return det



# Obtain the angle between two consecutive vectors (change in orientation, reorientation, turning angle).
# This angle is <0 if det<0 (clockwise) and >0 if counter clockwise
def reorientation(vec0,vec1):
    """ Given two vectors, obtain the change in the orientation (turning angle) in radians between 0 and 2·pi
        We consider a negative angle if the determinant is <0 (clockwise) and positive angle (det>0) if counter-clockwise
        
    Input:
        - two consecutive vectors vec0 and vec1
    
    Output:
        - reorientation angle (turning angle) between vectors in degrees (0 to 2·pi counter-clockwise and 0 to -2·pi clockwise)
    
    """
    
    unit_vec0=vec0/np.linalg.norm(vec0)
    unit_vec1=vec1/np.linalg.norm(vec1)
    dot_product=np.dot(unit_vec0,unit_vec1)
    a=np.arccos(dot_product)
    det=determinant(vec0,vec1)
    if det<0:
        return -a
    else:
        return a
    
    
    
def tortuosity(longitudes, latitudes):
    """ Given all the GPS coordenates (latitude and longitude) of the individual trajectory, obtain tortuosity of the path
    
    Input:
        - Lists of latitudes and longitudes of a single path
        
    Output:
        - Single value of the tortuosity of the path
        
    """

    latitude=df['latitude'].tolist()    # longitude and latitude to lists
    longitude=df['longitude'].tolist()

    reorientations=[]   # Re-orientation between the movement vector and the straight vector towards the final point.
    for i in range(1,len(latitude)-s):
        vectors=vector(latitude[i-1],longitude[i-1],latitude[i+s],longitude[i+s])  #movement vector
        vectors_straight=vector(latitude[i-1],longitude[i-1],latitude[-1],longitude[-1])  #straight vector
        reorientations.append(np.cos(abs(reorientation(vectors_straight,vectors)))) #reorientation
        reorientations2 = [x for x in reorientations if np.isnan(x) == False]  #avoid nan values

    efficiency=sum(reorientations2)/len(reorientations2)   # The orientation efficiency is <cos(0)>
    tortuosity=1.-efficiency        # The tortuosity is 1 minus the orientation efficiency. 

    return tortuosity
